In [ ]:
import warnings

warnings.filterwarnings('ignore')

## Constructing Circuits 

In this section we will explore building quantum circuits in pytket with the `Circuit` class and introduce some useful circuit primitives.

### Basic Bell circuit

In pytket we can create an instance of the `Circuit` class and add gates sequentially. We can prepare an entangled Bell state using a Hadamard and a CX gate

$$
\begin{equation}
\text{H} = \frac{1}{\sqrt{2}}
\begin{pmatrix}
1 & 1 \\
1 & -1
\end{pmatrix}\,, 
\quad
CX=
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{pmatrix}
\end{equation}
$$

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

bell_circ = Circuit(2, name="Bell")
bell_circ.H(0)
bell_circ.CX(0, 1)
render_circuit_jupyter(bell_circ)

In [ ]:
from pytket.extensions.qiskit import AerStateBackend

sv_backend = AerStateBackend()
result_state = sv_backend.run_circuit(bell_circ).get_state()
print("Statevector =", result_state)

Compare with the equation below...


$$
\begin{equation}
|\psi \rangle = \frac{1}{\sqrt{2}} \big(|00 \rangle + | 11 \rangle  \big)\,, \qquad \frac{1}{\sqrt{2}} \approx 0.707...
\end{equation}
$$

In [ ]:
from pytket.extensions.qiskit import AerBackend

bell_circ.measure_all()
render_circuit_jupyter(bell_circ)

backend = AerBackend()
result = backend.run_circuit(bell_circ, n_shots=1000)
print(result.get_counts())

In [ ]:
from plotting import plot_results

plot_results(result, dark_mode=True)

### Other Quantum Gates

* Some gates need to be accessed through the `OpType` enum
* Angles are specified as the number of half turns
* See the docs for all OpTypes - https://cqcl.github.io/tket/pytket/api/optype.html

## An Interesting Circuit - The Quantum Fourier Transform

As we've heard the Quantum Fourier Transform (QFT) is an important subroutine in several quantum algorithms including Shor's algorithm for factoring

We can build the circuit for the $n$ qubit QFT using $n$ Hadamard gates $\frac{n}{2}$ swap gates and $\frac{n(n-1)}{2}$ controlled unitary rotations. 

$$
\begin{equation}
\text{CU1} = 
\begin{pmatrix}
I & 0 \\
0 & \text{U1}
\end{pmatrix}
\,, \quad 
\text{U1} = 
\begin{pmatrix}
1 & 0 \\
0 & e^{i \pi \theta}
\end{pmatrix}
\end{equation}
$$

Note that in the code below we use the `OpType` enum to access the CU1 gate.

We will rotate by smaller and smaller angles of $\theta = \frac{1}{2^{n-1}}$ 

In [ ]:
# lets build the QFT for three qubits
from pytket.circuit import OpType

qft_circ = Circuit(3)

qft_circ.H(0)
qft_circ.add_gate(OpType.CU1 , [0.5], [1, 0])
qft_circ.add_gate(OpType.CU1 , [0.25], [2, 0])

qft_circ.H(1)
qft_circ.add_gate(OpType.CU1 , [0.5], [2, 1])

qft_circ.H(2)

qft_circ.SWAP(0, 2)

render_circuit_jupyter(qft_circ)

**Exercise:** Define a `build_qft_circ(n_qubits)` circuit that builds a circuit for the Quantum Fourier Transform for $n$ qubits.

### Circuit Boxes (I)

* Raw quantum gates are sometimes too low level to see whats going on
* We can wrap up sub circuits into boxes and add them to circuits (useful for subroutines).
* Oracle subroutine example below

Define a trivial oracle for Grover's algorithm: 

In [ ]:
oracle_circ = Circuit(4, name="Oracle")

oracle_circ.X(0).X(1).X(2).X(3)
oracle_circ.add_gate(OpType.CnZ, [0, 1, 2, 3])
oracle_circ.X(0).X(1).X(2).X(3)

render_circuit_jupyter(oracle_circ)

In [ ]:
from pytket.circuit import CircBox

oracle_box = CircBox(oracle_circ)
circ = Circuit(4)
circ.H(0).H(1).H(2).H(3) # Initial superposition
circ.add_circbox(oracle_box, [0, 1, 2, 3])

render_circuit_jupyter(circ)

### Circuit Boxes (II)

* Often we will need synthesise a quantum circuit to implement a gievn unitary
* Unitary synthesis is supported by TKET (Up to 3 qubits)
* **Fact:** A two qubit unitary can be implemented with at most 3 CX gates

Lets see an example with the Fermionic SWAP or $\text{FSWAP}$ gate

$$
\begin{equation}
\text{FSWAP} =
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0\\
0 & 0 & 0 & -1 
\end{pmatrix}
\end{equation}
$$

In [ ]:
from pytket.circuit import Unitary2qBox
import numpy as np

unitary_2q = np.asarray([
                 [1, 0, 0, 0],
                 [0, 0, 1, 0],
                 [0, 1, 0, 0],
                 [0, 0, 0, -1]])

u2_box = Unitary2qBox(unitary_2q)

test_circ = Circuit(2)
test_circ.add_unitary2qbox(u2_box, 0, 1)

render_circuit_jupyter(test_circ)

In [ ]:
from pytket.passes import DecomposeBoxes

DecomposeBoxes().apply(test_circ) # Decompose the box in place

render_circuit_jupyter(test_circ) # view raw circuit 

### Constructing a Circuit from QASM

* OpenQASM (Open Quantum assembly language provides a low level specification of quantum circuits)
* We Can easily build a circuit from a QASM file (And also export a circuit to QASM)

In [ ]:
from pytket.qasm import circuit_from_qasm

qpe_circ = circuit_from_qasm("qpe.qasm")
print("Total number of gates =", qpe_circ.n_gates)
print("#two qubit gates =", qpe_circ.n_2qb_gates())

render_circuit_jupyter(qpe_circ)

## Using Quantum devices and Simulators

In addition to the idealised noiseless simulators we used previously we can also target emulators which try to replicate the constraints and noise of a real quantum computer

In order to give some ideas of quantum compilation we can use a larger Quantum Phase estimation circuit and run it on an IBM Simulator.

**Disclaimer:** Not the most realistic experiment 

Visit https://quantum-computing.ibm.com/ to get an API token. Also see 

Credential saving process below
https://cqcl.github.io/pytket-qiskit/api/index.html#access-and-credentials

In [ ]:
#from qiskit import IBMQ
#IBMQ.save_account("MyToken", overwrite=True)

#from pytket.extensions.qiskit.backends.config import set_ibmq_config

#set_ibmq_config(ibmq_api_token="MyToken")

In [4]:
from pytket.extensions.qiskit import IBMQEmulatorBackend
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise.noise_model import NoiseModel
from pytket.qasm import circuit_from_qasm

qpe5_circ = circuit_from_qasm("qpe5.qasm")
noisy_backend = IBMQEmulatorBackend('ibmq_jakarta')


jakarta_noise_model = NoiseModel.from_backend(noisy_backend._ibmq._backend)

In [2]:
from qiskit_ibm_provider import IBMProvider

prov = IBMProvider()

noise_backend = prov.get_backend('ibmq_jakarta')

In [ ]:
noisy_aer_backend = AerBackend(jakarta_noise_model)

render_circuit_jupyter(qpe5_circ)

In [ ]:
from pytket.utils import Graph

Graph(qpe5_circ).get_qubit_graph()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

coupling_graph = noisy_aer_backend.backend_info.architecture.coupling
G = nx.Graph(coupling_graph)
plt.figure(figsize=(2,2))
nx.draw(G)
plt.show()

In [ ]:
compiled_circuit_l0 = noisy_aer_backend.get_compiled_circuit(qpe5_circ, optimisation_level=0)

cx_count = compiled_circuit_l0.n_2qb_gates()
print("How many CX gates do we have?" ,cx_count)

In [ ]:
render_circuit_jupyter(compiled_circuit_l0)


In [ ]:
Graph(compiled_circuit_l0).get_qubit_graph()

In [ ]:
handlel0 = noisy_aer_backend.process_circuit(compiled_circuit_l0, n_shots=1000)
resultl0 = noisy_aer_backend.get_result(handlel0)

In [ ]:
print(resultl0.get_counts().most_common()[0])

In [ ]:
compiled_qft = noisy_backend.get_compiled_circuit(qft_circ)

render_circuit_jupyter(compiled_qft)

In [ ]:
print(resultl0.get_counts().most_common()[0])
plot_results(resultl0, n_strings=4 , dark_mode=True)

In [ ]:
compiled_circuit_l2 = noisy_backend.get_compiled_circuit(qpe5_circ, optimisation_level=2)
cx_count = compiled_circuit_l2.n_gates_of_type(OpType.CX)
print(f"Our optimised circuit has {cx_count} CX gates")

handlel2 = noisy_aer_backend.process_circuit(compiled_circuit_l2, n_shots=1000)
resultl2 = noisy_aer_backend.get_result(handlel2)

In [ ]:
print(resultl2.get_counts().most_common()[0])

In [ ]:
plot_results(resultl2, n_strings=4, dark_mode=True)

### Switching Backends in pytket

* Researchers may want to explore running their quantum algorithms on different devices
* Uniform `Backend` interface for targeting different devices/simulators
* lets switch to a Cirq simulator. This simulator uses the {PhasedX, Rz, CZ} gatset

In [ ]:
from pytket.extensions.cirq import CirqStateSampleBackend

cirq_backend = CirqStateSampleBackend()
compiled_cirq = cirq_backend.get_compiled_circuit(qpe_circ)

render_circuit_jupyter(compiled_cirq)

cirq_result = cirq_backend.run_circuit(compiled_cirq, n_shots=1000)

## Brief section on Optimisation passes

* As we have seen with the phase estimation circuits we can boost the Fidelity of our circuits by reducing the number of gates in our circuit

* Lets try optimising a circuit from quantum chemistry with the general purpose `FullPeepholeOptimise` pass

In [ ]:
jw_chem_circ = circuit_from_qasm("H2JordanWignerMapper.qasm")
render_circuit_jupyter(jw_chem_circ)

print("Total gate count =", jw_chem_circ.n_gates)
print("#2 qubit gates =", jw_chem_circ.n_2qb_gates())

In [ ]:
from pytket.passes import FullPeepholeOptimise

FullPeepholeOptimise().apply(jw_chem_circ)

render_circuit_jupyter(jw_chem_circ)
print("Total gate count =", jw_chem_circ.n_gates)
print("#2 qubit gates =", jw_chem_circ.n_2qb_gates())

In [ ]:
from pytket.passes import auto_rebase_pass

ibm_gateset = {OpType.X, OpType.SX, OpType.Rz, OpType.CX}
ibm_rebase = auto_rebase_pass(ibm_gateset)

ibm_rebase.apply(jw_chem_circ)

The `auto_rebase_pass` works well if converting to a well known gateset i.e. one used by a hardware provider like IBM.

Rebases to other univeral sets of 1q and 2q gates can be accomplished with `RebaseCustom`.

In [ ]:
render_circuit_jupyter(jw_chem_circ)

We can also compose our own circuit transformations with `SequencePass`.

In [ ]:
from pytket.passes import SequencePass

seq_pass = SequencePass([FullPeepholeOptimise(), ibm_rebase])

jw_chem_circ = circuit_from_qasm("H2JordanWignerMapper.qasm")

seq_pass.apply(jw_chem_circ)

render_circuit_jupyter(jw_chem_circ)

### Working with Qiskit, Cirq and other libraries

* TKET supports easy conversion to and from qiskit circuits
* Also supported for (Cirq, pennylane, pyquil and Q# through the extensions)
* **Benefit:** Allows TKET compilation features to be used in conjunction with other tools 

In [ ]:
from pytket.extensions.qiskit import tk_to_qiskit #, qiskit_to_tk

qiskit_qc = tk_to_qiskit(qft_circ) 
# use qiskit_to_tk for conversion to TKET

qiskit_qc.draw('mpl', fold=-1)

## New Features!

## Arbitrary State Prepartation

We can prepare an arbitary quantum state in pytket. Note that the size of the circuit will scale exponentailly with the number of qubits

Lets prepare the following state

$$
\begin{equation}
    |W\rangle = \frac{1}{\sqrt{3}} \big(|001\rangle + |010\rangle +  |100\rangle \big)
\end{equation}
$$

In [ ]:
from pytket.circuit import StatePreparationBox

werner_state = 1 / np.sqrt(3) * np.array([0, 1, 1, 0, 1, 0, 0, 0])

werner_state_box = StatePreparationBox(werner_state)

state_circ = Circuit(3)
state_circ.add_gate(werner_state_box, [0, 1, 2])

render_circuit_jupyter(state_circ)

## ToffoliBox

A type of `Circuit` box that can implement an arbitary permutation of computational basis states.

Can "shuffle" the amplitudes of quantum state around as desired

$$
\begin{gather}
    |001\rangle \longmapsto |111\rangle \\
    |111\rangle \longmapsto |001\rangle \\
    |100\rangle \longmapsto |000\rangle \\
    |000\rangle \longmapsto |100\rangle
\end{gather}
$$

In [ ]:
from pytket.circuit import ToffoliBox

# Specify the desired permutation of the basis states
mapping = {
    (0, 0, 1): (1, 1, 1),
    (1, 1, 1): (0, 0, 1),
    (1, 0, 0): (0, 0, 0),
    (0, 0, 0): (1, 0, 0),
}

# Define box to perform the permutation
perm_box = ToffoliBox(permutation=mapping)

$$
\begin{equation}
\frac{1}{\sqrt{3}} \big(|001\rangle +  |010\rangle +|100\rangle \big)   \longmapsto \frac{1}{\sqrt{3}} \big(|000\rangle + |010\rangle + |111\rangle \big)
\end{equation}
$$

## ZX Diagrams

We have ZX diagrams in pytket. ZX is a graphical language for reasoning about quantum circuits

ZX has been used extensively in circuit simplification.

Read the [ZX Diagrams](https://cqcl.github.io/pytket/manual/manual_zx.html#) section of the user manual.


![ZX diagram](ZX_diagram.png)



## Summary - What have we learned?

* Overview of TKET and the challenges of running quantum algorithms
* Constructing circuits using the primitives available in pytket
* Running basic simulations of quantum circuits
* How to target different quantum devices with pytket
* Optimising circuits 
* Using TKET alongside other quantum SDKs


Install using the command 


<p><center> <code> pip install pytket </code> </center>

Useful links
* [API docs](https://cqcl.github.io/tket/pytket/api/)
* [Notebook examples](https://github.com/CQCL/pytket/tree/main/examples)
* [User Manual](https://cqcl.github.io/pytket/manual/manual_intro.html)

Visit the TKET repository -> https://github.com/CQCL/tket 

Open source  contributions welcome! :) 


## Any Questions?